<a href="https://colab.research.google.com/github/sitara12/My-projects/blob/main/bone_updated.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import zipfile
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
import tensorflow as tf
from tensorflow.keras import layers, models
from PIL import Image

# Specify the path to your dataset ZIP file
destination_path = '/content/BoneFractureDataset.zip'

# Specify the directory where you want to extract the dataset
extracted_dir = '/content/BoneFractureDataset'

# Extract the dataset
with zipfile.ZipFile(destination_path, 'r') as zip_ref:
    zip_ref.extractall(extracted_dir)

# Define functions for image preprocessing and label extraction
def preprocess_image(image_path):
    img = Image.open(image_path)
    img = img.resize((image_height, image_width))
    img = np.array(img) / 255.0
    return img

def extract_label_from_filename(filename):
    label = 1 if "fractured" in filename else 0
    return label

# Define image dimensions and number of channels
image_height, image_width, num_channels = 128, 128, 3

# Define the paths to your dataset folders
train_data_path = os.path.join(extracted_dir, 'train')
val_data_path = os.path.join(extracted_dir, 'val')

# Function to load data
def load_data(data_dir):
    image_data = []
    labels = []
    for label in os.listdir(data_dir):
        label_dir = os.path.join(data_dir, label)
        if os.path.isdir(label_dir):
            for image_file in os.listdir(label_dir):
                img = preprocess_image(os.path.join(label_dir, image_file))
                image_data.append(img)
                labels.append(extract_label_from_filename(image_file))
    return np.array(image_data), np.array(labels)

# Load training and validation data
X_train_images, y_train_images = load_data(os.path.join(train_data_path, 'fractured'))
X_train_images_non_fractured, y_train_images_non_fractured = load_data(os.path.join(train_data_path, 'non-fractured'))
X_val_images, y_val_images = load_data(os.path.join(val_data_path, 'fractured'))
X_val_images_non_fractured, y_val_images_non_fractured = load_data(os.path.join(val_data_path, 'non-fractured'))

# Combine fractured and non-fractured data
X_train_images = np.concatenate([X_train_images, X_train_images_non_fractured])
y_train_images = np.concatenate([y_train_images, y_train_images_non_fractured])
X_val_images = np.concatenate([X_val_images, X_val_images_non_fractured])
y_val_images = np.concatenate([y_val_images, y_val_images_non_fractured])

# Split data into training and testing sets
X_train_images, X_test_images, y_train_images, y_test_images = train_test_split(X_train_images, y_train_images, test_size=0.2, random_state=42)

# Standardize the data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train_images.reshape(-1, image_height * image_width * num_channels))
X_test = scaler.transform(X_test_images.reshape(-1, image_height * image_width * num_channels))

# Train the SVM model
svm_model = SVC()
svm_model.fit(X_train, y_train_images)

# Evaluate the SVM model on the test set
y_pred_svm = svm_model.predict(X_test)
svm_accuracy = accuracy_score(y_test_images, y_pred_svm)
print('SVM Accuracy:', svm_accuracy)

# Define the CNN model
model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(image_height, image_width, num_channels)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(1, activation='sigmoid')  # Assuming binary classification (fracture or not)
])

# Compile the CNN model
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Train the CNN model
model.fit(X_train_images, y_train_images, epochs=10, validation_data=(X_val_images, y_val_images))

# Evaluate the CNN model
test_loss, test_acc = model.evaluate(X_test_images, y_test_images)
print(f'CNN Test accuracy: {test_acc}')


BadZipFile: ignored